In [1]:
# --- Импорты ---
import pandas as pd
import numpy as np
import re
import time
import random
import json
import requests
from tqdm import tqdm
from typing import List, Dict
from sklearn.neighbors import NearestNeighbors
from sentence_transformers import SentenceTransformer
import hdbscan

# --- Параметры ---
CSV_PATH = 'data/stage_3_df_ru_region_skills_new_to_model_2.csv'
MODEL_NAME = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
EXCEL_OUT = 'vacancy_clusters.xlsx'
CHECKED_CSV_OUT = 'df_vac_checked_stage_3.csv'
MIN_POS_SIZE = 10
MAX_DISTANCE = 0.4
BATCH_SIZE = 15
REQUEST_DELAY = 10

API_URL = "https://api.mistral.ai/v1/chat/completions"
API_KEY = "1FyhNdyIvR4hR29waKXtRf4NaGAwl5Ru"
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

# --- 1. Препроцессинг вакансий ---
def normalize_title(title: str) -> str:
    if not isinstance(title, str) or not title.strip():
        return title

    level_pattern = r"(?i)\b(head of|middle|junior|стажер|intern|senior|lead|principal)\b"
    cleaned = re.sub(level_pattern, "", title)

    special_cases = {
        r'ux/ui': 'UX/UI', r'ui/ux': 'UI/UX',
        r'back[\-\u2013\u2014]?end': 'Backend', r'front[\-\u2013\u2014]?end': 'Frontend',
        r'full[\-\u2013\u2014]?stack': 'Fullstack'
    }

    for pattern, replacement in special_cases.items():
        cleaned = re.sub(pattern, replacement, cleaned, flags=re.IGNORECASE)

    cleaned = re.sub(r"(?<!\/)\/(?!\/)", " ", cleaned)
    cleaned = re.sub(r"[\-\u2013\u2014]", " ", cleaned)
    cleaned = " ".join(cleaned.split()).strip()

    return cleaned or title

# --- 2. Эмбеддинги ---
model = SentenceTransformer(MODEL_NAME)

def embed_texts(texts: List[str]) -> np.ndarray:
    return model.encode(texts, normalize_embeddings=True, show_progress_bar=False)

# --- 3. Кластеризация по position через HDBSCAN ---
def cluster_vacancies_by_position(df: pd.DataFrame, feature: str) -> Dict[str, pd.DataFrame]:
    result = {}

    for pos, group in tqdm(df.groupby('position'), desc="HDBSCAN кластеризация"):
        if len(group) < MIN_POS_SIZE:
            continue

        vacancies = group[feature].unique()
        embs = embed_texts(vacancies)

        clusterer = hdbscan.HDBSCAN(min_cluster_size=2, metric='euclidean')
        labels = clusterer.fit_predict(embs)

        canon_dict = {}
        for label in set(labels):
            if label == -1:
                continue
            cluster_vals = np.array(vacancies)[labels == label]
            canon = pd.Series(cluster_vals).value_counts().idxmax()
            for val in cluster_vals:
                canon_dict[val] = canon

        mapping = [{'canon': canon_dict.get(v, 'trash'), 'synonym': v} for v in vacancies]
        result[pos] = pd.DataFrame(mapping).drop_duplicates()

    return result

# --- 4. Проверка кластеров через Mistral ---
INSTRUCTION = """
Ты эксперт по IT-вакансиям. Проверь: является ли synonym_original синонимом canon_original.

Формат:
{
  "results": [
    {"correct_cluster": true/false},
    ...
  ]
}
"""

def prepare_prompt(batch: List[Dict]) -> List[Dict]:
    pairs = "\n".join([f"{i+1}. Канон: {row['canon']} | Синоним: {row['synonym']}" for i, row in enumerate(batch)])
    return [{"role": "system", "content": INSTRUCTION},
            {"role": "user", "content": f"Проверь пары:\n{pairs}\n\nФормат ответа: JSON"}]

def query_mistral(prompt: List[Dict]) -> dict:
    for attempt in range(3):
        try:
            res = requests.post(API_URL, headers=HEADERS, json={
                "model": "mistral-small-latest",
                "messages": prompt,
                "temperature": 0.1,
                "max_tokens": 900,
                "response_format": {"type": "json_object"}
            }, timeout=120)
            res.raise_for_status()
            content = res.json()['choices'][0]['message']['content'].strip()
            if content.startswith("```json"): content = content[7:-3].strip()
            content = content.replace(",}", "}").replace(",]", "]")
            return json.loads(content)
        except Exception as e:
            print(f"⚠️ Ошибка запроса Mistral (попытка {attempt+1}): {e}")
            time.sleep(3)
    return {}

def mistral_check(df_map: pd.DataFrame) -> pd.DataFrame:
    df_map = df_map.copy()
    df_map['mistral_check'] = False

    for i in tqdm(range(0, len(df_map), BATCH_SIZE), desc="Mistral проверка"):
        batch = df_map.iloc[i:i+BATCH_SIZE].to_dict('records')
        response = query_mistral(prepare_prompt(batch))

        if 'results' in response:
            for j, res in enumerate(response['results'][:len(batch)]):
                df_map.at[df_map.index[i + j], 'mistral_check'] = res.get('correct_cluster', False)
        else:
            print(f"⚠️ Пропущен батч: {i}-{i + BATCH_SIZE}")

        time.sleep(REQUEST_DELAY + random.random())

    return df_map

# --- 5. Главный пайплайн ---
def run_pipeline():
    df = pd.read_csv(CSV_PATH)
    df['vacancy_norm_1'] = df['vacancy it'].apply(normalize_title)

    pos_clusters = cluster_vacancies_by_position(df, feature='vacancy_norm_1')

    with pd.ExcelWriter(EXCEL_OUT) as writer:
        for pos, df_map in pos_clusters.items():
            df_map.to_excel(writer, sheet_name=pos[:30], index=False)

    checked_frames = []
    for pos, df_map in pos_clusters.items():
        print(f"🔍 Проверка позиции: {pos}")
        checked = mistral_check(df_map)
        checked_frames.append(checked)

    df_all_checked = pd.concat(checked_frames, ignore_index=True)
    valid_map = df_all_checked[df_all_checked['mistral_check']]
    replace_dict = dict(zip(valid_map['synonym'], valid_map['canon']))

    df['vacancy_norm_llm_check_1'] = df['vacancy_norm_1'].map(replace_dict).fillna(df['vacancy_norm_1'])

    with pd.ExcelWriter(EXCEL_OUT, mode="w") as writer:
        for pos, df_checked in zip(pos_clusters.keys(), checked_frames):
            df_checked.to_excel(writer, sheet_name=pos[:30], index=False)

    df.to_csv(CHECKED_CSV_OUT, index=False)
    print(f"✅ Готово! Данные сохранены в {EXCEL_OUT} и {CHECKED_CSV_OUT}")

# --- Запуск ---
if __name__ == "__main__":
    run_pipeline()


c:\Users\Aero-PC\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
HDBSCAN кластеризация: 100%|██████████| 24/24 [00:20<00:00,  1.16it/s]


🔍 Проверка позиции: BI-аналитик, аналитик данных


Mistral проверка: 100%|██████████| 9/9 [01:55<00:00, 12.86s/it]


🔍 Проверка позиции: DevOps-инженер


Mistral проверка: 100%|██████████| 5/5 [01:36<00:00, 19.38s/it]


...

🔍 Проверка позиции: Технический директор (CTO)


Mistral проверка: 100%|██████████| 4/4 [00:49<00:00, 12.27s/it]


🔍 Проверка позиции: Технический писатель


Mistral проверка: 100%|██████████| 4/4 [00:46<00:00, 11.60s/it]


✅ Готово! Данные сохранены в vacancy_clusters.xlsx и df_vac_checked_stage_3.csv
